In [1]:
import csv
import numpy as np

In [2]:
f = open('../Data/Preproceso/storms.csv')
csv_file = csv.reader(f)

In [3]:
cabeceras = csv_file.next()

In [4]:
episodios = dict()
for row in csv_file:
    epi = dict(zip(cabeceras, row))
    if row[11] in episodios:
        episodios[row[11]].append(epi)
    else:
        episodios[row[11]] = [epi]

In [5]:
f2 = open('../Data/Preproceso/locations.csv')
csv_file = csv.reader(f2)
cabeceras2 = csv_file.next()[1:]

In [6]:
locations = dict()
for row in csv_file:
    row = row[1:]
    loc = dict(zip(cabeceras2, row))
    if row[1] in locations:
        locations[row[1]].append(loc)
    else:
        locations[row[1]] = [loc]

In [7]:
locations_2 = dict()
for ep_id in locations:
    ids = []
    events = []
    ep = locations[ep_id]
    for evento in ep:
        if not evento['EVENT_ID'] in ids:
            ids.append(evento['EVENT_ID'])
            events.append(evento)
        locations_2[ep_id] = events

In [8]:
for loc_id in locations_2:
    loc = locations_2[loc_id]
    ep = episodios[loc_id]
    for evento1 in loc:
        event_id = evento1['EVENT_ID']
        for evento2 in ep:
            if evento2['EVENT_ID'] == event_id:
                evento2['RANGE'] = float(evento1['RANGE']) * 1.60934

In [9]:
import pandas as pd

In [10]:
lista = []
for ep_id in episodios:
    ep = episodios[ep_id]
    for event in ep:
        lista.append(event)

In [11]:
sumas = dict()

for ep_id in episodios:
    epi = episodios[ep_id]
    for evento in epi:
        if 'RANGE' in evento and evento['RANGE'] != '0.0':
            if evento['EVENT_TYPE'] in sumas:
                sumas[evento['EVENT_TYPE']].append(evento['RANGE'])
            else:
                sumas[evento['EVENT_TYPE']] = [evento['RANGE']]

In [12]:
medias = dict()
for suma_id in sumas:
    medias[suma_id] = np.average(sumas[suma_id])

In [13]:
for ep_id in episodios:
    epi = episodios[ep_id]
    for evento in epi:
        if not 'RANGE' in evento or evento['RANGE'] == 0:
            if evento['EVENT_TYPE'] in medias:
                evento['RANGE'] = medias[evento['EVENT_TYPE']]
            elif evento['EVENT_TYPE'] in ['Heavy Snow', 'Winter Storm']:
                evento['RANGE'] = medias['Heavy Rain']
            elif evento['EVENT_TYPE'] in ['Strong Wind', 'High Wind']:
                evento['RANGE'] = medias['Thunderstorm Wind']

In [14]:
lista = []
for ep_id in episodios:
    ep = episodios[ep_id]
    for event in ep:
        lista.append(event)

df = pd.DataFrame(lista)
df = df.set_index('EVENT_ID')
df.to_csv('../Data/Preproceso/storms_final_preproceso.csv')